In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
# Execute request
# If you’re using a different site just replace the url e.g. r=requests.get(‘put your url in here’)
r = requests.get('https://www.yelp.com/biz/tesla-san-francisco?osq=Tesla+Dealership')

In [4]:
r.status_code

200

In [5]:
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')

In [7]:
divs = soup.findAll(class_="raw__09f24__T4Ezm")

In [8]:
reviews = []
for div in divs:
    reviews.append(div.text)
    

In [9]:
del reviews[0:5]
reviews

["Best service center. The team is very responsive, knowledgeable & friendly. I've visited for glass, filters, tires, front upper control arms, radar & more; always high-quality & lasting repairs, with clear explanations provided.",
 'I went here today to get my front two tires replaced. Tesla\'s are amazing vehicles, certainly I don\'t have any problems with the vehicle itself. It\'s the men working the tire department. I went there and my whole time being there I felt pressured and forced. First an older guy came up to me and checked my front passenger tire out and said it would last a little while more, so a replacement wasn\'t necessary. Then another gentleman came and was extremely hyper, literally running around and tweaking the whole time. Dude came and checked all 4 of my tires in like 10 seconds max and said I needed 4 tires. Which I agreed with him because I myself knew I needed all 4 to be replaced. Then I\'m assuming the "manager" there was also extremely forceful and made 

In [10]:
for i in range(1, 8):
    r2 = requests.get(f'https://www.yelp.com/biz/tesla-san-francisco?osq=Tesla+Dealership&start={i*10}')
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    divs2 = soup2.findAll(class_="raw__09f24__T4Ezm")
    reviews2 = []
    for div2 in divs2:
        reviews2.append(div2.text)
    del reviews2[0:5]
    reviews = reviews+reviews2
    
print('end of loop')

end of loop


In [11]:
reviews[0]

"Best service center. The team is very responsive, knowledgeable & friendly. I've visited for glass, filters, tires, front upper control arms, radar & more; always high-quality & lasting repairs, with clear explanations provided."

# Analysing The Data

In [12]:
import  pandas as pd
import  numpy as np

In [13]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [14]:
df.head()

,review
0,Best service center. The team is very responsi...
1,I went here today to get my front two tires re...
2,Went here for repair after noticing a little g...
3,Delivery and customer service experience is be...
4,Slow and inhospitable. It should never take 3 ...


In [15]:
len(df['review'])

91

In [16]:
df['word_count'] = df['review'].apply(lambda x: len(x.split()))

In [17]:
df.head()

,review,word_count
0,Best service center. The team is very responsi...,33
1,I went here today to get my front two tires re...,273
2,Went here for repair after noticing a little g...,132
3,Delivery and customer service experience is be...,51
4,Slow and inhospitable. It should never take 3 ...,25


In [18]:
df['char_count'] = df['review'].apply(lambda x: len(x))

In [19]:
def average_words(x):
    words = x.split()
    return sum(len(word) for word in words) / len(words)

In [20]:
df['average_word_length'] = df['review'].apply(lambda x: average_words(x))

In [21]:
df.head()

,review,word_count,char_count,average_word_length
0,Best service center. The team is very responsi...,33,228,5.939394
1,I went here today to get my front two tires re...,273,1472,4.395604
2,Went here for repair after noticing a little g...,132,742,4.628788
3,Delivery and customer service experience is be...,51,296,4.823529
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000


In [24]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
stop_words = stopwords.words('english')

In [26]:
len(stop_words)

179

In [27]:
df['stopword_count'] = df['review'].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))

In [28]:
df['stopword_rate'] = df['stopword_count']/df['word_count']

In [29]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515
1,I went here today to get my front two tires re...,273,1472,4.395604,126,0.461538
2,Went here for repair after noticing a little g...,132,742,4.628788,61,0.462121
3,Delivery and customer service experience is be...,51,296,4.823529,24,0.470588
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000,9,0.360000


In [30]:
df.sort_values(by='stopword_rate')

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515
79,23.9 miles away from Tesla,5,26,4.400000,1,0.200000
67,23.9 miles away from Tesla,5,26,4.400000,1,0.200000
46,23.9 miles away from Tesla,5,26,4.400000,1,0.200000
34,23.9 miles away from Tesla,5,26,4.400000,1,0.200000
...,...,...,...,...,...,...
44,During my first time I went into the San Franc...,541,2786,4.151571,292,0.539741
66,We are currently doing our research into the p...,276,1465,4.311594,150,0.543478
14,Today I went to the San Francisco Tesla Dealer...,121,621,4.140496,66,0.545455
41,Finally my model x is ready after three month ...,321,1542,3.806854,176,0.548287


In [31]:
df.describe()

,word_count,char_count,average_word_length,stopword_count,stopword_rate
count,91.000000,91.000000,91.000000,91.000000,91.000000
mean,119.736264,651.527473,4.500522,56.835165,0.439818
std,105.270322,562.192718,0.366076,52.650897,0.092754
min,5.000000,26.000000,3.384615,1.000000,0.151515
25%,46.500000,267.000000,4.311797,20.000000,0.423657
50%,86.000000,468.000000,4.450000,41.000000,0.465625
75%,158.500000,855.500000,4.656018,74.000000,0.493632
max,541.000000,2786.000000,5.939394,292.000000,0.550000


# Data Cleaning

In [32]:
df['lowercase'] = df['review'].apply(lambda x: " ".join(word.lower() for word in x.split()))

In [33]:
df['punctuation'] = df.lowercase.str.replace('[^\w\s]', '')

<ipython-input-33-d76f9e5baaee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['punctuation'] = df.lowercase.str.replace('[^\w\s]', '')


In [34]:
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [35]:
stop_words = stopwords.words('english')

In [36]:
df['stopwords'] = df['punctuation'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [37]:
pd.Series(" ".join(df['stopwords']).split()).value_counts()[:30]

car            130
service        115
tesla          104
get             54
model           40
would           36
back            32
experience      32
customer        31
time            31
drive           30
one             30
us              28
new             27
told            27
appointment     27
said            24
like            24
even            23
test            21
call            19
im              19
go              19
take            18
went            18
3               18
day             18
location        18
center          18
first           18
dtype: int64

In [38]:
other_stop_words = ['get','told','would','week','us','test','right','left','one','even','also','go','asked']

In [39]:
len(other_stop_words)

13

In [40]:
df['cleanreview'] = df['stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stop_words))

In [41]:
#pd.Series(" ".join(df['cleanreview']).split()).value_counts()[:30]
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515,best service center. the team is very responsi...,best service center the team is very responsiv...,best service center team responsive knowledgea...,best service center team responsive knowledgea...
1,I went here today to get my front two tires re...,273,1472,4.395604,126,0.461538,i went here today to get my front two tires re...,i went here today to get my front two tires re...,went today get front two tires replaced teslas...,went today front two tires replaced teslas ama...
2,Went here for repair after noticing a little g...,132,742,4.628788,61,0.462121,went here for repair after noticing a little g...,went here for repair after noticing a little g...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...
3,Delivery and customer service experience is be...,51,296,4.823529,24,0.470588,delivery and customer service experience is be...,delivery and customer service experience is be...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000,9,0.360000,slow and inhospitable. it should never take 3 ...,slow and inhospitable it should never take 3 h...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hours replace t...


# Lemmatization

In [42]:
! pip install -U textblob


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
#import textblob
import textblob            #to import
from textblob import Word
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [48]:
df['lemmatized'] = df['cleanreview'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

#df['lemmatized_word_count'] = df['lemmatized'].apply(lambda x: len([word for word in x.split()))

In [ ]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview,lemmatized
0,I took back my 2018 Model 3 last month for saf...,49,270,4.530612,15,0.306122,i took back my 2018 model 3 last month for saf...,i took back my 2018 model 3 last month for saf...,took back 2018 model 3 last month safety recal...,took back 2018 model 3 last month safety recal...,took back 2018 model 3 last month safety recal...
1,"My model 3 got punctured on monday, slow leak ...",113,609,4.398230,48,0.424779,"my model 3 got punctured on monday, slow leak ...",my model 3 got punctured on monday slow leak f...,model 3 got punctured monday slow leak 2 days ...,model 3 got punctured monday slow leak 2 days ...,model 3 got punctured monday slow leak 2 day d...
2,In a nutshell: Tesla sucks! I leased one of th...,136,734,4.404412,65,0.477941,in a nutshell: tesla sucks! i leased one of th...,in a nutshell tesla sucks i leased one of thei...,nutshell tesla sucks leased one model ys 2021 ...,nutshell tesla sucks leased model ys 2021 tech...,nutshell tesla suck leased model y 2021 techni...
3,Nick has been amazing in educating us about th...,83,459,4.542169,41,0.493976,nick has been amazing in educating us about th...,nick has been amazing in educating us about th...,nick amazing educating us teslas answering man...,nick amazing educating teslas answering many q...,nick amazing educating tesla answering many qu...
4,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515,best service center. the team is very responsi...,best service center the team is very responsiv...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,best service center team responsive knowledgea...


# Sentiment Analysis

In [49]:
from textblob import TextBlob

In [50]:
df['polarity'] = df['lemmatized'].apply(lambda  x: TextBlob(x).sentiment[0])

In [51]:
df['subjectivity'] = df['lemmatized'].apply(lambda x: TextBlob(x).sentiment[1])

In [52]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview,lemmatized,polarity,subjectivity
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515,best service center. the team is very responsi...,best service center the team is very responsiv...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,0.229167,0.213889
1,I went here today to get my front two tires re...,273,1472,4.395604,126,0.461538,i went here today to get my front two tires re...,i went here today to get my front two tires re...,went today get front two tires replaced teslas...,went today front two tires replaced teslas ama...,went today front two tire replaced tesla amazi...,0.013309,0.510693
2,Went here for repair after noticing a little g...,132,742,4.628788,61,0.462121,went here for repair after noticing a little g...,went here for repair after noticing a little g...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,0.023864,0.403030
3,Delivery and customer service experience is be...,51,296,4.823529,24,0.470588,delivery and customer service experience is be...,delivery and customer service experience is be...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,-0.075000,0.728472
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000,9,0.360000,slow and inhospitable. it should never take 3 ...,slow and inhospitable it should never take 3 h...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hour replace ti...,-0.150000,0.700000


In [ ]:
#df.drop(['lowercase','punctuation','stopwords','cleanreview','lemmatized'],axis=1, inplace=True)

In [53]:
df.describe()

,word_count,char_count,average_word_length,stopword_count,stopword_rate,polarity,subjectivity
count,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,119.736264,651.527473,4.500522,56.835165,0.439818,0.096848,0.463317
std,105.270322,562.192718,0.366076,52.650897,0.092754,0.246806,0.200322
min,5.000000,26.000000,3.384615,1.000000,0.151515,-1.000000,0.000000
25%,46.500000,267.000000,4.311797,20.000000,0.423657,-0.010227,0.408182
50%,86.000000,468.000000,4.450000,41.000000,0.465625,0.072261,0.485606
75%,158.500000,855.500000,4.656018,74.000000,0.493632,0.253279,0.593750
max,541.000000,2786.000000,5.939394,292.000000,0.550000,0.800000,1.000000


In [54]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview,lemmatized,polarity,subjectivity
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515,best service center. the team is very responsi...,best service center the team is very responsiv...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,0.229167,0.213889
1,I went here today to get my front two tires re...,273,1472,4.395604,126,0.461538,i went here today to get my front two tires re...,i went here today to get my front two tires re...,went today get front two tires replaced teslas...,went today front two tires replaced teslas ama...,went today front two tire replaced tesla amazi...,0.013309,0.510693
2,Went here for repair after noticing a little g...,132,742,4.628788,61,0.462121,went here for repair after noticing a little g...,went here for repair after noticing a little g...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,0.023864,0.403030
3,Delivery and customer service experience is be...,51,296,4.823529,24,0.470588,delivery and customer service experience is be...,delivery and customer service experience is be...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,-0.075000,0.728472
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000,9,0.360000,slow and inhospitable. it should never take 3 ...,slow and inhospitable it should never take 3 h...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hour replace ti...,-0.150000,0.700000


In [55]:
df.sort_values(by='polarity')

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview,lemmatized,polarity,subjectivity
72,Horrible service. Felt swindled. No customer c...,39,221,4.692308,19,0.487179,horrible service. felt swindled. no customer c...,horrible service felt swindled no customer car...,horrible service felt swindled customer care d...,horrible service felt swindled customer care d...,horrible service felt swindled customer care d...,-1.000000,1.000000
73,Tesla service is awful. So much worse than Sub...,71,420,4.929577,31,0.436620,tesla service is awful. so much worse than sub...,tesla service is awful so much worse than suba...,tesla service awful much worse subaru example ...,tesla service awful much worse subaru example ...,tesla service awful much worse subaru example ...,-0.435000,0.680000
40,The service department is horrible in getting ...,72,374,4.125000,38,0.527778,the service department is horrible in getting ...,the service department is horrible in getting ...,service department horrible getting loaner car...,service department horrible getting loaner car...,service department horrible getting loaner car...,-0.400000,0.662500
53,I had a bad experience. Technician names Adam ...,57,320,4.631579,28,0.491228,i had a bad experience. technician names adam ...,i had a bad experience technician names adam w...,bad experience technician names adam rude arro...,bad experience technician names adam rude arro...,bad experience technician name adam rude arrog...,-0.280000,0.386667
75,"Oh, I forgot to mention the most shocking of a...",71,381,4.380282,34,0.478873,"oh, i forgot to mention the most shocking of a...",oh i forgot to mention the most shocking of al...,oh forgot mention shocking interior trim left ...,oh forgot mention shocking interior trim sides...,oh forgot mention shocking interior trim side ...,-0.270833,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,There is inconsistency of excellent service he...,67,372,4.492537,32,0.477612,there is inconsistency of excellent service he...,there is inconsistency of excellent service he...,inconsistency excellent service super duper ha...,inconsistency excellent service super duper ha...,inconsistency excellent service super duper ha...,0.451042,0.720833
10,Nick has been amazing in educating us about th...,83,459,4.542169,41,0.493976,nick has been amazing in educating us about th...,nick has been amazing in educating us about th...,nick amazing educating us teslas answering man...,nick amazing educating teslas answering many q...,nick amazing educating tesla answering many qu...,0.502066,0.640220
52,Wow!Test driving of a Tesla was superb! It's ...,79,482,5.088608,34,0.430380,wow!test driving of a tesla was superb! it's i...,wowtest driving of a tesla was superb its inno...,wowtest driving tesla superb innovative cruisi...,wowtest driving tesla superb innovative cruisi...,wowtest driving tesla superb innovative cruisi...,0.584722,0.811111
71,"Visited OLiver, and had nothing but the best d...",39,218,4.615385,18,0.461538,"visited oliver, and had nothing but the best d...",visited oliver and had nothing but the best de...,visited oliver nothing best dealings purchased...,visited oliver nothing best dealings purchased...,visited oliver nothing best dealing purchased ...,0.750000,0.400000


In [56]:
df['subjectivity']

0     0.213889
1     0.510693
2     0.403030
3     0.728472
4     0.700000
        ...   
86    0.647802
87    0.497300
88    0.272727
89    0.300000
90    0.000000
Name: subjectivity, Length: 91, dtype: float64

In [58]:
df.loc[0]

review                 Best service center. The team is very responsi...
word_count                                                            33
char_count                                                           228
average_word_length                                             5.939394
stopword_count                                                         5
stopword_rate                                                   0.151515
lowercase              best service center. the team is very responsi...
punctuation            best service center the team is very responsiv...
stopwords              best service center team responsive knowledgea...
cleanreview            best service center team responsive knowledgea...
lemmatized             best service center team responsive knowledgea...
polarity                                                        0.229167
subjectivity                                                    0.213889
Name: 0, dtype: object

In [60]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

In [62]:
import collections

In [84]:
badreviews= df.loc[df['polarity']<=0]
allbadreviews=" "

for s in badreviews['review']:
  spaces=" "+s
  allbadreviews+=spaces

commoncausedict= word_count(allbadreviews)

sortedreasons = reversed(sorted(commoncausedict.items(), key=lambda item: item[1]))
#
c=0
for k,v in sortedreasons:
  if k not in stop_words and k not in other_stop_words:
    print(k,v)
    c+=1
  if c==200:
    break

#sortedreasons.iteritems() 


I 75
car 32
Tesla 26
service 24
The 11
car. 8
like 8
miles 8
back 8
never 8
experience 8
customer 8
I'm 7
He 7
away 7
23.9 7
order 7
They 7
find 6
service. 6
This 6
bad 6
called 6
call 6
take 6
person 5
- 5
guy 5
month 5
first 5
Tesla. 5
name 4
weeks 4
day 4
took 4
time 4
new 4
made 4
poor 4
drive 4
waiting 4
two 4
need 4
pick 4
come 4
tell 4
Do 4
charge 4
After 4
Model 4
3 4
another 4
trim 3
Subaru 3
side 3
interior 3
purchase 3
work 3
We 3
people 3
No 3
sign 3
seemed 3
me, 3
check 3
different 3
appointment 3
schedule 3
part 3
advise 3
Redwood 3
say 3
said 3
3.0 3
AP 3
And 3
refused 3
model 3
department 3
multiple 3
It's 3
fix 3
want 3
If 3
app 3
phone 3
(and 3
service, 3
cars 3
fit 3
center 3
put 3
company 3
hold 3
But 3
& 3
entire 3
issues 3
tire 3
hours 3
ever 3
lot 3
horrible 3
wood 2
Carbon 2
covered 2
makes 2
co-workers 2
end 2
major 2
front 2
battery 2
quality 2
picked 2
turn 2
however 2
expect 2
similar 2
leaving 2
negative 2
Marin 2
Ness 2
Van 2
Outside". 2
"Park 2
Their 2
NO

In [82]:
badreviews

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleanreview,lemmatized,polarity,subjectivity
0,Best service center. The team is very responsi...,33,228,5.939394,5,0.151515,best service center. the team is very responsi...,best service center the team is very responsiv...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,best service center team responsive knowledgea...,0.229167,0.213889
1,I went here today to get my front two tires re...,273,1472,4.395604,126,0.461538,i went here today to get my front two tires re...,i went here today to get my front two tires re...,went today get front two tires replaced teslas...,went today front two tires replaced teslas ama...,went today front two tire replaced tesla amazi...,0.013309,0.510693
2,Went here for repair after noticing a little g...,132,742,4.628788,61,0.462121,went here for repair after noticing a little g...,went here for repair after noticing a little g...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,went repair noticing little gap rocker panel p...,0.023864,0.403030
3,Delivery and customer service experience is be...,51,296,4.823529,24,0.470588,delivery and customer service experience is be...,delivery and customer service experience is be...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,delivery customer service experience beyond ho...,-0.075000,0.728472
4,Slow and inhospitable. It should never take 3 ...,25,136,4.480000,9,0.360000,slow and inhospitable. it should never take 3 ...,slow and inhospitable it should never take 3 h...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hours replace t...,slow inhospitable never take 3 hour replace ti...,-0.150000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,This location is my favorite! I've been here ...,85,468,4.505882,41,0.482353,this location is my favorite! i've been here t...,this location is my favorite ive been here twi...,location favorite ive twice service customer s...,location favorite ive twice service customer s...,location favorite ive twice service customer s...,0.321703,0.647802
87,As opposed to the location in Palo Alto Stanfo...,174,960,4.459770,83,0.477011,as opposed to the location in palo alto stanfo...,as opposed to the location in palo alto stanfo...,opposed location palo alto stanford shopping c...,opposed location palo alto stanford shopping c...,opposed location palo alto stanford shopping c...,0.144406,0.497300
88,"While I do enjoy my Tesla, the customer servic...",115,619,4.391304,58,0.504348,"while i do enjoy my tesla, the customer servic...",while i do enjoy my tesla the customer service...,enjoy tesla customer service nonexistent writt...,enjoy tesla customer service nonexistent writt...,enjoy tesla customer service nonexistent writt...,0.050000,0.272727
89,This dealership is an extension of Elon Musk's...,92,535,4.826087,42,0.456522,this dealership is an extension of elon musk's...,this dealership is an extension of elon musks ...,dealership extension elon musks person arrogan...,dealership extension elon musks person arrogan...,dealership extension elon musk person arrogant...,0.216667,0.300000


In [70]:
allbadreviews

'  23.9 miles away from Tesla Dear dad can you buy me a Tesla to drive at grad school! 23.9 miles away from Tesla 23.9 miles away from Tesla 23.9 miles away from Tesla 23.9 miles away from Tesla 23.9 miles away from Tesla 23.9 miles away from Tesla'

In [64]:
commoncausedict

{}